In [ ]:
!pip install -q transformers torch datasets pandas scikit-learn accelerate bitsandbytes peft fuzzywuzzy python-levenshtein

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time
import re
from fuzzywuzzy import fuzz 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
with open('categories.txt', 'r', encoding='utf-8') as f:
    categories = [line.strip() for line in f.readlines()]

print(f"Категории: {categories}")
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# Осмотр train
print("\nПримеры из train:")
print(train_df['text'].head(3).tolist())

# Проверка на NaN и дубликаты
print(f"NaN в train: {train_df.isnull().sum().sum()}")
train_df.drop_duplicates(subset=['text'], inplace=True)
print(f"Train после удаления дубликатов: {train_df.shape}")

In [ ]:
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'\s+', ' ', text.strip())
    text = re.sub(r'[^\w\s.,!?—–-]', ' ', text)
    if len(text) < 10:
        return ""
    return text

# Применение предобработки
train_df['text_clean'] = train_df['text'].apply(preprocess_text)
test_df['text_clean'] = test_df['text'].apply(preprocess_text)

# Удаление пустых после очистки
train_df = train_df[train_df['text_clean'] != ''].reset_index(drop=True)
test_df = test_df[test_df['text_clean'] != ''].reset_index(drop=True)

print(f"Train после предобработки: {train_df.shape}")
print("\nПримеры очищенных текстов:")
print(train_df['text_clean'].head(3).tolist())

In [ ]:
# Модель для разметки 
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

device = "cuda" if torch.cuda.is_available() else "cpu"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto" if device == "cuda" else None,
    trust_remote_code=True
)

print(f"Модель загружена на {device}")

In [ ]:
few_shot_examples = """
Пример 1: "Футболка хорошего качества, но велика." -> одежда
Пример 2: "Кроссовки не подошли по размеру." -> обувь
Пример 3: "Холодильник шумит сильно." -> бытовая техника
Пример 4: "Тарелки красивые, но бьются." -> посуда
Пример 5: "Полотенце мягкое, приятное." -> текстиль
Пример 6: "Игрушка для ребенка сломалась." -> товары для детей
Пример 7: "Серьги подделка, цвет не тот." -> украшения и аксессуары
Пример 8: "Телефон не заряжается." -> электроника
Пример 9: "Заказ не пришел, деньги не вернули." -> нет товара
"""

def classify_review(text, model, tokenizer, categories, few_shot=few_shot_examples):
    prompt = f"""Ты классификатор отзывов по категориям товаров. Используй только одну категорию из списка: {', '.join(categories)}.
{few_shot}
Отзыв: "{text}"
Категория:"""
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred = generated.split("Категория:")[-1].strip().lower()
    
    # Fuzzy matching для лучшей точности
    best_match = max(categories, key=lambda cat: fuzz.ratio(pred, cat.lower()))
    confidence = fuzz.ratio(pred, best_match.lower()) / 100.0
    
    return best_match if confidence > 0.6 else "нет товара"  

# Тест на одном примере
test_text = train_df['text_clean'].iloc[0]
pred = classify_review(test_text, base_model, tokenizer, categories)
print(f"Отзыв: {test_text[:100]}...")
print(f"Предсказание: {pred}")

In [ ]:
batch_size = 50
train_labeled = []

start_time_total = time.time()
for i in range(0, len(train_df), batch_size):
    batch = train_df['text_clean'][i:i+batch_size]
    batch_preds = []
    
    batch_start = time.time()
    for text in batch:
        pred = classify_review(text, base_model, tokenizer, categories)
        batch_preds.append(pred)
    
    train_labeled.extend(batch_preds)
    elapsed_batch = time.time() - batch_start
    print(f"Батч {i//batch_size + 1}: {len(batch)} примеров за {elapsed_batch:.2f}с (ср. {elapsed_batch/len(batch):.2f}с/пример)")
    torch.cuda.empty_cache() if device == "cuda" else None

total_time = time.time() - start_time_total
print(f"Общее время разметки: {total_time:.2f}с (ср. {total_time/len(train_df):.2f}с/пример)")

train_df_labeled = train_df.copy()
train_df_labeled['label'] = train_labeled
train_df_labeled[['text_clean', 'label']].to_csv('train_labeled.csv', index=False)
print(f"Сохранено в train_labeled.csv: {train_df_labeled.shape}")
print("\nРаспределение классов:")
print(train_df_labeled['label'].value_counts())

In [ ]:
print("Распределение по категориям:")
label_counts = train_df_labeled['label'].value_counts()
print(label_counts)


for cat in categories[:3]:  
    examples = train_df_labeled[train_df_labeled['label'] == cat]['text_clean'].head(1).tolist()
    if examples:
        print(f"\n{cat}: {examples[0][:100]}...")

print("\nГотово! Теперь можно перейти к fine-tune в отдельном notebook'e, используя train_labeled.csv")